In [ ]:
import os
import sys
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import DataLoader

WORK_DIR = Path(Path.cwd()).parent
sys.path.append(str(WORK_DIR))
from src.datasets import get_dataset, get_dataloader
from src.utils import parse_data_cfg, IMG, FPHA, LMDB, DATA_DIR, YOLO

In [ ]:
cfgname = 'exp1/yolov2_fpha_hpo_bbox_data_mod1_noflip'
cfg = parse_data_cfg(WORK_DIR/'data_cfg'/(cfgname + '.cfg'))
epoch = 200
exp_dir = cfg["exp_dir"]
data_split = 'test'
split_set = cfg[data_split + '_set']

# Visualize Dataloader

In [ ]:
dataset_kwargs = {'split_set': split_set}
dataset   = get_dataset(cfg, dataset_kwargs)
sampler   = None
shuffle   = cfg['shuffle']
kwargs = {'batch_size'  :   int(cfg['batch_size']),
          'shuffle'     :   shuffle,
          'num_workers' :   int(cfg['num_workers']),
          'pin_memory'  :   True}
data_loader = get_dataloader(dataset, sampler, kwargs)

In [ ]:
idx = 0 
for i, (img, bbox_gt, uvd_gt) in enumerate(data_loader):
    if i == idx:
        batch_size = img.shape[0]
        img = img.cpu().numpy()
        img = np.swapaxes(img, 2, 3)
        img = np.swapaxes(img, 1, 3)
        img = IMG.scale_img_255(img)
        bbox_gt = bbox_gt.cpu().numpy()
        uvd_gt = uvd_gt.cpu().numpy()
        uvd_gt = np.squeeze(uvd_gt)
        break
    i += 1

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
idx = 0
for i in range(4):
    for j in range(4):
        k = np.ravel_multi_index((i, j), (4, 4))
        if k >= len(img):
            break
        cur_img = img[idx]
        b = bbox_gt[idx]
        u = uvd_gt[idx]        
        ax[i, j].imshow(cur_img)
        FPHA.draw_bbox(ax[i, j], b, (cur_img.shape[1], cur_img.shape[0]))
        u = IMG.scale_points_WH(u, (1,1), (cur_img.shape[0], cur_img.shape[1]))
        FPHA.visualize_joints_2d(ax[i, j], u[FPHA.REORDER_IDX], joint_idxs=False)
        idx += 1

# Evaluation

In [ ]:
keys = LMDB.get_keys(os.path.join(DATA_DIR, split_set + "_keys_cache.p"))
bbox_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + "_bbox_gt.lmdb"), "float32", 4)
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_bbox.txt'.format(epoch, data_split))
bbox_pred = np.loadtxt(pred_file)
bbox_pred = np.asarray([bbox[:5] for bbox in bbox_pred]) # take only the first bbox

xyz_gt = LMDB.read_all_lmdb_dataroot(keys, os.path.join(DATA_DIR, split_set + '_xyz_gt.lmdb'), 'float32', (21, 3))
uvd_gt = FPHA.xyz2uvd_color(xyz_gt)
pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_best.txt'.format(epoch, data_split))
pred_uvd = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_uvd = IMG.scale_points_WH(pred_uvd, (1, 1), (1920, 1080))
pred_uvd[..., 2] *= 1000
pred_xyz = FPHA.uvd2xyz_color(pred_uvd)

pred_file = os.path.join(DATA_DIR, exp_dir, 'predict_{}_{}_conf.txt'.format(epoch, data_split))
pred_conf = np.loadtxt(pred_file)

In [ ]:
idx = 0
print(keys[idx])

img = np.asarray(Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx])))
fig, ax = plt.subplots()
ax.imshow(img)

FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]), 'r')
FPHA.visualize_joints_2d(ax, pred_uvd[idx][FPHA.REORDER_IDX], joint_idxs=False, c='r')
FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')
FPHA.visualize_joints_2d(ax, uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False, c='b')

fig, ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(img)
ax[0].set_title('pred')
FPHA.visualize_joints_2d(ax[0], pred_uvd[idx][FPHA.REORDER_IDX], joint_idxs=False)
ax[1].imshow(img)
ax[1].set_title('true')
FPHA.visualize_joints_2d(ax[1], uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(221)
ax.imshow(img)
FPHA.visualize_joints_2d(ax, pred_uvd[idx][FPHA.REORDER_IDX], joint_idxs=False, c='r')
FPHA.visualize_joints_2d(ax, uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False, c='b')

for proj_idx, (proj_1, proj_2) in enumerate([[0, 1], [1, 2], [0, 2]]):
    ax = fig.add_subplot(2, 2, 2 + proj_idx)
    if proj_idx == 0:
        # Invert y axes to align with image in camera projection
        ax.invert_yaxis()
    ax.set_aspect('equal')
    FPHA.visualize_joints_2d(ax,
                        np.stack(
                            [pred_uvd[idx][FPHA.REORDER_IDX][:, proj_1], 
                             pred_uvd[idx][FPHA.REORDER_IDX][:, proj_2]],
                            axis=1),
                        joint_idxs=False, c='r')
    FPHA.visualize_joints_2d(ax,
                        np.stack(
                            [uvd_gt[idx][FPHA.REORDER_IDX][:, proj_1], 
                             uvd_gt[idx][FPHA.REORDER_IDX][:, proj_2]],
                            axis=1),
                        joint_idxs=False, c='b')    

In [ ]:
max_idx = []
# get the best idx for each 2D cell
for i in range(len(pred_conf[idx])//5):
    max_idx.append(i*5 + np.argmax(pred_conf[idx][i*5:i*5+5]))

fig, ax = plt.subplots(figsize=(5,5))
pred_uvd_416 = IMG.scale_points_WH(pred_uvd[idx], (1920, 1080), (416, 416))
FPHA.visualize_joints_2d(ax, pred_uvd_416[FPHA.REORDER_IDX], joint_idxs=False)
img_rsz = IMG.resize_img(img, (416, 416))
ax.imshow(img_rsz.astype('uint32'))

# red is the best
# yellow is anything over 0.9
import matplotlib.patches as patches
for i in range(len(max_idx)):
    index = np.unravel_index(i, (13, 13))
    x = index[1]
    y = index[0]
    al = pred_conf[idx][max_idx[i]]
    if al == np.amax(pred_conf[idx]):
        c = 'r'
    elif al <= 0.8:
        c = 'b'
    else:
        c = 'y'
    rect = patches.Rectangle((x*32,y*32),32,32,linewidth=1, edgecolor=c, facecolor=c, fill=True, alpha=al)
    ax.add_patch(rect)

In [ ]:
iou_thresh = 0.5
correct = 0
avg_iou = 0
worst_iou = 10
worst_iou_idx = 0
iou_list = []
for i in range(len(bbox_pred)):
    iou = YOLO.bbox_iou(bbox_pred[i], bbox_gt[i])
    avg_iou += iou
    iou_list.append(iou)
    if iou < worst_iou:
        worst_iou = iou
        worst_iou_idx = i
        
    if iou > iou_thresh:
        correct += 1
        
recall = correct/len(bbox_pred)
avg_iou = avg_iou/len(bbox_pred)
print('Recall:', recall, 'Avg_IOU:', avg_iou)
print('Worst Bbox id:', worst_iou_idx)
for idx in np.argsort(iou_list):
    print(idx)

In [ ]:
print('%s UVD mean_l2_error: ' %data_split, FPHA.mean_L2_error(uvd_gt[:len(pred_uvd)], pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, FPHA.mean_L2_error(xyz_gt[:len(pred_uvd)], pred_xyz))
error = []
for i, (pred, uvd) in enumerate(zip(pred_uvd, uvd_gt)):
#     print(i, FPHA.mean_L2_error(uvd, pred))
    error.append(FPHA.mean_L2_error(uvd, pred))
error = np.asarray(error)
min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)
print('Best Pose id:', min_error_idx, 'uvd_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uvd_error:', error[max_error_idx])
for idx in np.argsort(error):
    print(idx)

In [ ]:
pck = FPHA.percentage_frames_within_error_curve(xyz_gt[:len(pred_uvd)], pred_xyz)
pck_str = ''
for p in pck:
    pck_str += str(p) + ', '
print(pck_str)
thresholds = np.arange(0, 85, 5)
print('AUC:', FPHA.calc_auc(pck, thresholds))

# Create GIF

In [ ]:
from moviepy.editor import ImageSequenceClip
from tqdm import tqdm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

# seq = 'Subject_6/prick/3'
# seq = 'Subject_2/stir/3'
# seq = 'Subject_5/open_liquid_soap/3'
# seq = 'Subject_5/toast_wine/1'
seq = 'Subject_1/tear_paper/3'
# seq = 'Subject_2/charge_cell_phone/1'
# seq = 'Subject_3/squeeze_paper/3'
SAVE_DIR = Path(DATA_DIR)/'acv-data'/'gifs'
seq_keys_list = [(i, k.split('/')) for i, k in enumerate(keys) if seq in k]

index_list, seq_list = zip(*seq_keys_list)

seq_list = [int(i[-1].split('_')[-1].split('.')[0]) for i in seq_list]
ind = np.argsort(seq_list).astype('uint32')

index_list = np.asarray(index_list)
index_list = index_list[ind]

frames = []
xyz_gt_vid = []
pred_xyz_vid = []
for idx in tqdm(index_list):
    idx = int(idx)
    img = np.asarray(Image.open(os.path.join(DATA_DIR, 'First_Person_Action_Benchmark', 'Video_files', keys[idx])))

    fig, ax = plt.subplots()
    ax = fig.gca()
    ax.imshow(img)
    ax.axis('off')
    FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]), 'r')
    FPHA.visualize_joints_2d(ax, pred_uvd[idx][FPHA.REORDER_IDX], joint_idxs=False, c='r')
    FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')
    FPHA.visualize_joints_2d(ax, uvd_gt[idx][FPHA.REORDER_IDX], joint_idxs=False, c='b')
    fig.canvas.draw()
    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(data)
    
    xyz_gt_vid.append(FPHA.uvd2xyz_color(uvd_gt[idx]))
    pred_xyz_vid.append(FPHA.uvd2xyz_color(pred_uvd[idx]))
    
    plt.close() 
    
segment_clip = ImageSequenceClip(frames, fps=6)
name = SAVE_DIR/('{}_{}_{}_{}.gif'.format(cfgname, epoch, data_split, seq.replace('/', '_')))
segment_clip.write_gif(name, fps=6)
from IPython.display import Image as IPythonImage
with open(name,'rb') as f:
    display(IPythonImage(data=f.read(), format='png'))

In [ ]:
pck = FPHA.percentage_frames_within_error_curve(np.asarray(xyz_gt_vid), np.asarray(pred_xyz_vid))
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', FPHA.calc_auc(pck, thresholds))